## UnboundLocalError

Laut ChatGPT...

Der Fehler `UnboundLocalError` tritt auf, wenn in einer Funktion auf eine lokale Variable zugegriffen wird, BEVOR dieser ein Wert zugewiesen wurde. Dies geschieht typischerweise, wenn Python eine Variable innerhalb einer Funktion als lokal interpretiert, weil sie irgendwo in der Funktion zugewiesen wird, aber an einer Stelle darauf zugegriffen wird, bevor die Zuweisung stattgefunden hat.

### Prüfungsaufgabe:

[Fehler-Code Pythontutor](https://pythontutor.com/render.html#code=x%20%3D%2042%0A%20%20%20%0Adef%20test%28%29%3A%20%0A%20%20x%20%3D%20x%20%25%204%0A%20%20print%28x%29%0A%20%20%20%0Atest%28%29%0Aprint%28x%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

In [97]:
x = 42
   
def test():
  x = x % 4
  print(x)
   
test()
   
print(x)

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

#### Antwortmöglichkeiten:

1. 10 2
2. Error   # Richtige Lösung
3. 42 10
4. 2 42    # Meine Wahl

### Analyse:

Wenn man sich diesen Code anschaut könnte man folgendes denken:

In [10]:
x = 42
   
def test(): # 2. Die Funktion wird aufgeruf, was auch ohne Argument möglich ist.
  x = x % 4 # 3. Die Funktion HAT ZUGRIFF auf die globale Variable x = 42. Siehe Test 1. unten. 
  print(x)  # 4. Warum kommt es zu einem UnboundLocalError?
     
test() # 1. Python ruft die Funktion "test()" auf.
   
print(x)

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

Zur besseren Lesbarkeit habe ich die Variablen in `x1` und `x2` umbenannt.

In [ ]:
x = 42 

def test():
  x1 = x2 % 4 
  print(x1)
     
test()
   
print(x)

Ich denke, dass Python den Code wie folgt interpretiert:                                                                   
Weise der Variable `x1` den Wert zu, der beim Ausdruck `(x2 % 4)` herauskommt.                                                  
Und `x2` hat teoretisch Zugriff auf auf die globale Variable
`x = 42` haben.                                                                                                    
Aber aus irgendeinem Grund wird ein Fehler geworfen, BEVOR Python prüft ob es die globale Variable `x = 42` gibt... 

Oder Python sieht die globale Variable `x = 42` aber priorisiert die Variable `x` im Namespace der Funktion, selbst wenn diese nicht definiert ist und zu einem Fehler führt...

Lösung von ChatGPT:

Innerhalb der Funktion `test()` wird versucht, die Variable `x` zu verwenden und zu verändern, BEVOR sie innerhalb des Funktionsscopes definiert wurde. Durch die unterschiedlichen Namensräume weiß Python nicht, dass es `x` aus der globalen Variable `x = 42` verwenden soll, wenn diese Variable nicht exlizit beim Funktionsaufruf als Argument übergeben wird. Das führt zu einem UnboundLocalError.

### Vermutung:

In [ ]:
x = 42 

def test():
  x = x % 4 
  print(x)
     
test()
   
print(x)

Python sucht offenbar zuerst in der Funktion selbst nach x. Er findet x = x % 4, merkt dass diese Gleichung nicht gelöst werden kann, da ein Wert fehlt der x zugewiesen wird und wirft den UnboundLocalError.

oder

Entweder Python beginnt eine Rekursive Endlosschleife, stellt dann fest, dass das nicht funktioniert und wirft dann den UnboundLocalError.

```python
x = x % 4 also setzt Python für x --> x % 4 ein x = (x % 4) % 4
    
x = ((((x % 4) % 4) % 4) % 4) % 4 etc.
```

Rekursive Endlosschleife die in einem Fehler endet. Allerdings würde man eher den StackoverflowError erwarten, oder wenigstens RuntimeError... 

### Neue Frage:

Was ich nicht verstehe ist, warum der Code keinen UnboundLokalError mehr wirft, wenn man die Variable `x = x % 4` durch `y = x % 4` ersetzt.

Die Variable die Probleme macht, dass `x` in `x % 4` hat sich doch garnicht verändert... 

### Tests in denen Verschiedene Dinge untersucht werden

#### Test 1.

Können Funktionen auf globale Variablen zugreifen?

In [6]:
a = 2
b = 3

def test():
    c = a + b
    print(c)

test()

5


Ja, natürlich können Funktionen auf globale Variablen zugreifen...

#### Test 2.

Was würde passieren, wenn die Variablen erst nach dem Aufruf der Funktion definiert werden?

In [2]:
def test():
    c = a + b
    print(c)

test()

a = 2
b = 3

NameError: name 'a' is not defined

Hier entsteht kein UnboundLocalError, sondern ein NameError...
Es gibt also einen Enscheidenden Unterschied zwischen diesen beiden Fehlerklassen. Aber welchen?

#### Test 3.

Vergleich mit geänderter Variable um zu beweisen, dass es sich um unterschiedliche Namensräume handelt.

Aber dann stellt sich die Frage, ob der Namespace überhaupt etwas mit dem UnboundLocalError zutun hat...?

In [29]:
y = 42 # Jup, komplett unterschiedliche Namensräume / Variablen
    
def test():
  x = x % 4 
  print(x)
     
test()
   
print(x)

UnboundLocalError: cannot access local variable 'x' where it is not associated with a value

#### Test 4.

Vermutlich gibt es irgendeinen Interne Regel in Python, das erst innerhalb einer Funktion nach Variablen gesucht wird, die für die Funktion verwendet werden können, bevor man auf globale Variablen zurückgreift.  

In [6]:
y = 42
    
def test():
    x = 10 # Wenn x klar als locale Variable definiert wurde gibt es keine Probleme.
    x = x % 4 # x = 10 --> 10 % 4 = 2 --> das Ergebnis wird der lokalen Variable x zugewiesen.
    print(x) # Die neue lokale Variable x wird gedruckt.
     
test()
   
print(y)

2
42


## Lösung:

Um den Fehler zu beheben, kann man:

#### Option 1.

Die Verwendung einer anderen lokalen Variable

In [14]:
x = 42

def test():
    y = x % 4  # Eine neue lokale Variable y wird verwendet und irgendwie greift x hier auf x = 42 
    print(y) # Dieser print greift auf die lokale Variable y zu

test()
print(x)

2
42


[Code im Pythontutor](https://pythontutor.com/render.html#code=x%20%3D%2042%0A%20%20%20%0Adef%20test%28%29%3A%20%0A%20%20y%20%3D%20x%20%25%204%0A%20%20print%28y%29%0A%20%20%20%0Atest%28%29%0Aprint%28x%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)

Vermutlich kommt es in diesem Fall nicht mehr zu einer Verwechslung zwischen dem `"x" = x % 4` und `"x" = 42`. Python hat innerhalb der Funktion keine Defintion der Variable `x` mehr und greift deshalb brav auf die globale Variable zurück.

#### Option 2.

x in der test() x als Parameter definieren und dann als Argument übergeben.

In [20]:
x = 42
   
def test(x): # 2. Die Funktion wird aufgeruf, was auch ohne Argument möglich ist, (wenn auch nicht Sinnvoll)
  x = x % 4 # 3. Man sollte meinen, dass x % 4 Zugriff auf die Variable x = 42 hat, 
  print(x)  # aber offenbar sind das zwei Unterschiedliche durch den Namespace getrennte Variablen.
     
test(x) # 1. Python ruft die Funktion "test()" auf
print(x)

2
42


Zur besseren Lesbarkeit habe ich x in x1 umbennant.

In [7]:
x1 = 42

def test(x1):
    x = x1 % 4  
    print(x) 

test(x1)
print(x1)

2
42


Dieser Code ist einhervorragendes Beispiel dafür warum man nicht jede Variable x nennen sollte!